In [ ]:
# ! pip install wandb
# ! pip install pydot
# ! pip install graphviz
# ! pip install datasets
# ! pip install scikit-learn

# Importing necessary Libraries

In [ ]:
import os
import shutil
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from datasets import load_from_disk
from IPython.display import Image
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense, Dropout, GlobalAveragePooling2D, Input,
                                     InputLayer, MaxPool2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.utils import plot_model
from wandb.keras import WandbCallback

import wandb

In [ ]:
PROJECT_SUFFIX = "frequency_classifier_multi"
ENTITY = "makersplace"
PROJECT = f"ai-or-not-{PROJECT_SUFFIX}"
SEED = 7
RUNTIME_DATE_SUFFIX = "%m%d_%H%M"

# current time
JOB_TYPE_SUFFIX = f"{PROJECT_SUFFIX}_M"
RUN_NAME_SUFFIX = datetime.now().strftime(RUNTIME_DATE_SUFFIX)


# Datasets Paths
training_dataset_path = "../cache/data/training_dataset"
validation_dataset_path = "../cache/data/validation_dataset"

# Model Paths
cnn_model_path = Path(f"../cache/models/{JOB_TYPE_SUFFIX}/cnn_{RUN_NAME_SUFFIX}")
effv2_model_dir_path = Path(f"../cache/models/{JOB_TYPE_SUFFIX}/en2s_{RUN_NAME_SUFFIX}")


# Deleted and recreated training and validation dataset folders
CLEAN_RUN = False


np.random.seed(SEED)
tf.random.set_seed(SEED)

# WANDB Login
os.environ["WANDB_API_KEY"] = "d13afab09b400fc9d606e612d806a4b0740790fd"
wandb.login()

# Configuration

In [ ]:
CONFIGURATION = {
    "BATCH_SIZE": 64,
    "IM_SIZE": 128,
    "DROPOUT_RATE": 0.1,
    "N_EPOCHS": 15,
    "REGULARIZATION_RATE": 0.01,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 2048,
    "N_DENSE_2": 1024,
    "N_DENSE_3": 256,
    "LEARNING_RATE": 0.001,
    "CHANNELS": 3,
    "CLASS_NAMES": ["REAL", "ADM", "SDv1", "MD"],
}

# DataSet Configuration

In [ ]:
TRAIN_DIRECTORIES = [
    # Label 0
    ("DIRE/train/imagenet/real", 0, "*/*"),
    ("DIRE/train/celebahq/real", 0, "*"),
    ("DIRE/train/lsun_bedroom/real", 0, "*"),
    ("cifake/train/REAL", 0, "*"),
    # Label 1
    ("DIRE/train/imagenet/adm", 1, "*/*"),
    ("DIRE/train/lsun_bedroom/adm", 1, "*"),
    # Label 2
    ("cifake/train/FAKE", 2, "*"),  # Generated by SD 1.4
    (
        "FakeImageDataset/ImageData/train/SDv15R-CC1M/SDv15R-dpmsolver-25-1M/SDv15R-CC1M",
        2,
        "*",
    ),  # Generated by SD 1.5
    ("DIRE/train/celebahq/sdv2", 2, "*/*"),
    # Label 3
    ("FakeImageDataset/ImageData/val/Midjourneyv5-5K/Midjourneyv5-5K_train", 3, "*"),
]
# Randomize the order of the training directories
# np.random.shuffle(train_directories)

In [ ]:
# Test Directories
TEST_SHARDS = 1
test_directories = [
    # Label 0
    ("../cache/data/cifake/test/REAL", 0, "*", TEST_SHARDS, 0),
    ("../cache/data/DIRE/test/imagenet/real", 0, "*/*", TEST_SHARDS, 0),
    ("../cache/data/DIRE/test/celebahq/real", 0, "*", TEST_SHARDS, 0),
    # Label 1
    ("../cache/data/DIRE/test/imagenet/adm", 1, "*/*", TEST_SHARDS, 0),
    # Label 2
    ("../cache/data/cifake/test/FAKE", 2, "*", TEST_SHARDS, 0),  # Generated by SD 1.4
    ("../cache/data/DIRE/test/imagenet/sdv1", 2, "*/*", TEST_SHARDS, 0),  # Bad 73
    ("../cache/data/DIRE/test/lsun_bedroom/sdv1_new", 2, "*", TEST_SHARDS, 0),
    (
        "../cache/data/FakeImageDataset/ImageData/val/SDv15-CC30K/SDv15-CC30K",
        2,
        "*/*",
        6,
        0,
    ),
    # Label 3
    ("../cache/data/DIRE/test/lsun_bedroom/sdv2", 2, "*", TEST_SHARDS, 0),
    ("../cache/data/DIRE/test/celebahq/sdv2", 2, "*", TEST_SHARDS, 0),
    (
        "../cache/data/FakeImageDataset/ImageData/val/SDv21-CC15K/SDv21-CC15K/SDv2-dpmsolver-25-10K",
        2,
        "*",
        2,
        0,
    ),  # Bad 79
    # Label 4
    (
        "../cache/data/FakeImageDataset/ImageData/val/Midjourneyv5-5K/Midjourneyv5-5K_test",
        3,
        "*",
        TEST_SHARDS,
        0,
    ),  # Bad  65
    (
        "../cache/data/DIRE/test/lsun_bedroom/midjourney",
        3,
        "*",
        TEST_SHARDS,
        0,
    ),  # Bad < 13
    # # AI Artbench Dataset
    # ("../cache/data/ai-artbench/test/AI*", 1.0, "*", TEST_SHARDS, 0),  # 675 Batches
    # ("../cache/data/ai-artbench/test/real", 0.0, "*/*", TEST_SHARDS, 0),
    # # CIFAKE Dataset
    # ("../cache/data/FakeImageDataset/ImageData/val/cogview2-22K/cogview2-22K", 1.0, "*", TEST_SHARDS, 0),
    # DIRE Imagenet Dataset
    # ("../cache/data/DIRE/test/celebahq/if", 1.0, "*", TEST_SHARDS, 0),
    # ("../cache/data/DIRE/test/celebahq/dalle2", 1.0, "*", TEST_SHARDS, 0),
    # # DIRE Lsun Bedroom Dataset
    # ("../cache/data/DIRE/test/lsun_bedroom/dalle2", 1.0, "*", TEST_SHARDS, 0),
    # ("../cache/data/DIRE/test/lsun_bedroom/vqdiffusion", 1.0, "*", TEST_SHARDS, 0),
    # FakeImageDataset
]

# Dataset Loading and Tranformations

In [ ]:
def resize_image(image):
    image = tf.image.resize_with_pad(
        image=image,
        target_height=CONFIGURATION["IM_SIZE"],
        target_width=CONFIGURATION["IM_SIZE"],
    )
    # divide by 255 to normalize
    image = image / 255.0

    return image


def decode_img(img):
    img = tf.io.decode_image(img, channels=3)
    return resize_image(img)


def process_path(file_path):
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img


# write a function apply Fourier Transform to the image and return the image
def apply_fourier_transform(image):
    # print(f"Image shape: {image.shape}")
    # extract r channel from the image
    r = image[:, :, 0]
    # extract g channel from the image
    g = image[:, :, 1]
    # extract b channel from the image
    b = image[:, :, 2]

    # apply fourier transform to the image
    r = tf.signal.fft2d(tf.cast(r, dtype=tf.complex64))
    g = tf.signal.fft2d(tf.cast(g, dtype=tf.complex64))
    b = tf.signal.fft2d(tf.cast(b, dtype=tf.complex64))
    # # shift the zero-frequency component to the center of the spectrum
    r = tf.signal.fftshift(r)
    g = tf.signal.fftshift(g)
    b = tf.signal.fftshift(b)
    # apply log to the image enhance the magnitude of the image and to reduce the dynamic range of the data for visualization
    r = 20 * tf.math.log(tf.abs(r) + 1)
    g = 20 * tf.math.log(tf.abs(g) + 1)
    b = 20 * tf.math.log(tf.abs(b) + 1)
    # normalize the value using min-max normalization
    r = (r - tf.reduce_min(r)) / (tf.reduce_max(r) - tf.reduce_min(r))
    g = (g - tf.reduce_min(g)) / (tf.reduce_max(g) - tf.reduce_min(g))
    b = (b - tf.reduce_min(b)) / (tf.reduce_max(b) - tf.reduce_min(b))
    # merge channels
    if CONFIGURATION["CHANNELS"] == 6:
        o_r = image[:, :, 0]
        o_g = image[:, :, 1]
        o_b = image[:, :, 2]
        image = tf.stack([o_r, o_g, o_b, r, g, b], axis=-1)
    else:
        image = tf.stack([r, g, b], axis=-1)

    return image


def visualize_dataset(samples):
    plt.figure(figsize=(12, 12))
    index = 1
    for image, label in samples:
        plt.subplot(4, 4, index)
        plt.imshow(image)
        title = CONFIGURATION["CLASS_NAMES"][int(label)]
        plt.title(title)
        plt.axis("off")
        index += 1

    plt.show()


def get_custom_dataset(directory, label, pattern):
    # if directory path contains 'aiornot' load it as tf dataset else load it as a custom dataset
    if "aiornot" in directory:
        read_aiornot = load_from_disk(dataset_path=directory)
        dataset = read_aiornot.to_tf_dataset(
            columns="image",
            label_cols="label",
        )
        dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32), y))
        dataset = dataset.map(lambda x, y: (resize_image(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    else:
        list_ds = tf.data.Dataset.list_files(str(Path(directory) / pattern), shuffle=True)
        dataset = list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.map(lambda x: (x, label))

    return dataset

# Individual Dataset Creation and Visualization

In [ ]:
# declare empty dataset for trainingn and validation
training_dataset = None
validation_dataset = None

for directory, label, pattern, shards, shard_index in train_directories:
    print(
        f"""
**********************************************************************************************************************************          
          Directory:  {directory} 
**********************************************************************************************************************************
    """
    )

    current_dataset = get_custom_dataset(directory, label, pattern)

    if shards > 1:
        # shard the dataset
        current_dataset = current_dataset.shard(num_shards=shards, index=shard_index)

    # split dataset into train and validation
    dataset_size = len(current_dataset)
    train_size = int(0.8 * dataset_size)
    validation_size = int(0.2 * dataset_size)

    current_train_dataset = current_dataset.skip(validation_size)
    current_validation_dataset = current_dataset.take(validation_size)

    if training_dataset is None:
        training_dataset = current_train_dataset
    else:
        training_dataset = training_dataset.concatenate(current_train_dataset)

    if validation_dataset is None:
        validation_dataset = current_validation_dataset
    else:
        validation_dataset = validation_dataset.concatenate(current_validation_dataset)

    # VISUALIZE DATASET
    visualize_dataset(current_dataset.take(16))

In [ ]:
# apply fourier transform to the image
training_dataset = training_dataset.map(lambda x, y: (apply_fourier_transform(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (apply_fourier_transform(x), y))

In [ ]:
training_dataset = (
    training_dataset.shuffle(buffer_size=20_000, seed=SEED)
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
)

validation_dataset = validation_dataset.batch(CONFIGURATION["BATCH_SIZE"]).prefetch(tf.data.AUTOTUNE)


# visualize_dataset(training_dataset.take(16))

In [ ]:
if CLEAN_RUN:
    if os.path.exists(training_dataset_path):
        # remove directory recursively
        shutil.rmtree(training_dataset_path)

    if os.path.exists(validation_dataset_path):
        shutil.rmtree(validation_dataset_path)

if not os.path.exists(training_dataset_path):
    training_dataset.save(training_dataset_path)

if not os.path.exists(validation_dataset_path):
    validation_dataset.save(validation_dataset_path)

In [ ]:
# load dataset sets from disk
training_dataset = tf.data.Dataset.load(training_dataset_path)
validation_dataset = tf.data.Dataset.load(validation_dataset_path)

# Training Data Visualization

In [ ]:
plt.figure(figsize=(12, 12))
for image, label in training_dataset.take(1):
    for index in range(16):
        ax = plt.subplot(4, 4, index + 1)
        # extract red channel from the image
        plt.imshow(image[index][:, :, 0])
        # plt.imshow(image[index])
        plt.title(CONFIGURATION["CLASS_NAMES"][int(label[index])])
        plt.axis("off")

# Callbacks for Model

In [ ]:
# Reduce LR On no Improvement
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=1,
    mode="min",
    min_delta=0.001,
    cooldown=0,
    min_lr=1e-15,
)

# Early Stopping
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=8,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
effv2_model_dir_path.mkdir(parents=True, exist_ok=True)
effv2_model_best_weights_path = effv2_model_dir_path / "best_weights"
effv2_model_saved_model_path = effv2_model_dir_path / "saved_model"


wandb_run = wandb.init(
    entity=ENTITY,
    project=PROJECT,
    job_type=f"effv2s_{JOB_TYPE_SUFFIX}",
    name=f"fc_layers_{RUN_NAME_SUFFIX}",
    reinit=True,
    config=CONFIGURATION,
    settings=wandb.Settings(start_method="fork"),
)

# Model Definition

In [ ]:
# Model Checkpointing
model_checkpoint = ModelCheckpoint(
    effv2_model_best_weights_path,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min",
    save_freq="epoch",
)

backbone = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(
        CONFIGURATION["IM_SIZE"],
        CONFIGURATION["IM_SIZE"],
        CONFIGURATION["CHANNELS"],
    ),
)

backbone.trainable = True

In [ ]:
efficientnetv2s_model = tf.keras.Sequential(
    [
        Input(
            shape=(
                CONFIGURATION["IM_SIZE"],
                CONFIGURATION["IM_SIZE"],
                CONFIGURATION["CHANNELS"],
            )
        ),
        backbone,
        GlobalAveragePooling2D(),
        Dense(
            CONFIGURATION["N_DENSE_1"],
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"]),
        ),
        BatchNormalization(),
        Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),
        Dense(
            CONFIGURATION["N_DENSE_2"],
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"]),
        ),
        BatchNormalization(),
        Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),
        Dense(
            CONFIGURATION["N_DENSE_3"],
            activation="relu",
            kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"]),
        ),
        BatchNormalization(),
        Dense(len(CONFIGURATION["CLASS_NAMES"]), activation="softmax"),
    ]
)

efficientnetv2s_model.summary()

In [ ]:
plot_model(
    efficientnetv2s_model,
    to_file="efficientnet_b4_model.png",
    show_shapes=True,
    show_layer_names=True,
)
Image(filename="efficientnet_b4_model.png")

# Model Training

In [ ]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

efficientnetv2s_model.compile(
    optimizer=Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=metrics,
)

efficientnetv2s_history = efficientnetv2s_model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=CONFIGURATION["N_EPOCHS"] * 3,
    callbacks=[
        reduce_lr,
        model_checkpoint,
        early_stopping,
        WandbCallback(save_model=False),
    ],
)

# Model Evaluation

In [ ]:
def log_classification_report(model, dataset, wandb_run):
    y_true = []
    y_pred = []
    for image, label in dataset:
        y_true.extend(label.numpy())

    y_pred = model.predict(dataset)
    y_pred_labels = np.argmax(y_pred, axis=1)

    report = classification_report(
        y_true,
        y_pred_labels,
        target_names=CONFIGURATION["CLASS_NAMES"],
        output_dict=True,
    )

    avg_report_df = pd.DataFrame()
    avg_report_df["model"] = [wandb_run.name]
    avg_report_df["Avg precision"] = [report["macro avg"]["precision"]]
    avg_report_df["Avg recall"] = [report["macro avg"]["recall"]]
    avg_report_df["Avg F1"] = [report["macro avg"]["f1-score"]]
    avg_report_df["Avg accuracy"] = [report["accuracy"]]
    avg_report_df["Avg support"] = [report["macro avg"]["support"]]

    label_reoprt_df = pd.DataFrame()
    label_reoprt_df["model"] = [wandb_run.name]
    for class_name in CONFIGURATION["CLASS_NAMES"]:
        label_reoprt_df[f"{class_name} P/R/F1"] = (
            f"{report[class_name]['precision']:.2f} / {report[class_name]['recall']:.2f} / {report[class_name]['f1-score']:.2f}"
        )

    wandb_run.log({"Classification Report": wandb.Table(dataframe=avg_report_df)})
    wandb_run.log({"Label Report": wandb.Table(dataframe=label_reoprt_df)})
    wandb_run.log(
        {
            "confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=y_true,
                preds=y_pred_labels,
                class_names=CONFIGURATION["CLASS_NAMES"],
            )
        }
    )


def log_test_metrics(current_model, wandb_run):
    # declare empty dataframe and add results to it for each test dataset
    loss_dataframe = pd.DataFrame()
    accuracy_dataframe = pd.DataFrame()

    # composite dataframe
    composite_dataframe = pd.DataFrame()

    loss_dataframe["model"] = [wandb_run.name]
    accuracy_dataframe["model"] = [wandb_run.name]
    # instantiate empty dataset
    composite_dataset = None

    # itreate through each test dataset using index
    for index, (directory, label, pattern, shards, shard_index) in enumerate(test_directories):
        current_dataset = get_custom_dataset(directory, label, pattern)
        current_dataset = current_dataset.map(lambda x, y: (apply_fourier_transform(x), y))
        # current_dataset.ignore_errors()
        current_dataset = current_dataset.batch(CONFIGURATION["BATCH_SIZE"])

        # evaluate model on current_dataset and capture metrics
        # accept only the loss and accuracy and ignore the rest irrespective of count of metrics
        test_loss, test_accuracy = current_model.evaluate(current_dataset)

        # Extract the 3rd part of the directory path
        dataset_name = directory.split("/")[3] + " : " + directory.split("/")[-1] + " : " + str(index)

        # add a column to the dataframe
        loss_dataframe[dataset_name] = [test_loss]
        accuracy_dataframe[dataset_name] = [test_accuracy]

        # concatenate current dataset to composite dataset
        if composite_dataset is None:
            composite_dataset = current_dataset
        else:
            composite_dataset = composite_dataset.concatenate(current_dataset)

    composite_loss, composite_accuracy = current_model.evaluate(composite_dataset)
    composite_dataframe["model"] = [wandb_run.name]
    composite_dataframe["loss"] = [composite_loss]
    composite_dataframe["accuracy"] = [composite_accuracy]

    wandb_run.log({"Loss": wandb.Table(dataframe=loss_dataframe)})
    wandb_run.log({"Accuracy": wandb.Table(dataframe=accuracy_dataframe)})
    wandb_run.log({"Overall Results": wandb.Table(dataframe=composite_dataframe)})

    log_classification_report(current_model, composite_dataset, wandb_run)

In [ ]:
efficientnetv2s_model.load_weights(effv2_model_best_weights_path)
efficientnetv2s_model.save(effv2_model_saved_model_path)
efficientnetv2s_model.evaluate(validation_dataset)

log_test_metrics(efficientnetv2s_model, wandb_run)

wandb_run.finish()

In [ ]:
wandb_run.finish()
plt.plot(efficientnetv2s_history.history["loss"])
plt.plot(efficientnetv2s_history.history["val_loss"])
plt.title("EfficientNetV2 S Model Loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(["train_loss", "val_loss"])
plt.show()

In [ ]:
plt.plot(efficientnetv2s_history.history["sparse_categorical_accuracy"])
plt.plot(efficientnetv2s_history.history["val_sparse_categorical_accuracy"])
plt.title("EfficientNetV2 S Model Accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(["train_accuracy", "val_accuracy"])
plt.show()

In [ ]:
# # write a function to print confusion matrix
# def confusion_matrix(y_true, y_pred, labels):
#     cm = tf.math.confusion_matrix(y_true, y_pred, num_classes=len(labels))
#     cm = cm.numpy()
#     cm = pd.DataFrame(cm, index=labels, columns=labels)
#     return cm

# composite_dataset = None
#  # itreate through each test dataset using index
# for index, (directory, label, pattern, shards, shard_index) in enumerate(test_directories):

#     current_dataset = get_custom_dataset(directory, label, pattern)
#     current_dataset = current_dataset.map(lambda x, y: (apply_fourier_transform(x), y))
#     # current_dataset.ignore_errors()
#     current_dataset = current_dataset.batch(CONFIGURATION["BATCH_SIZE"])

#     # concatenate current dataset to composite dataset
#     if composite_dataset is None:
#         composite_dataset = current_dataset
#     else:
#         composite_dataset = composite_dataset.concatenate(current_dataset)

# y_pred = efficientnetv2s_model.predict(composite_dataset)
# y_pred_labels = np.argmax(y_pred, axis=1)

# y_true = []
# for image, label in composite_dataset:
#     y_true.extend(label.numpy())

# print(confusion_matrix(y_true, y_pred_labels, CLASS_NAMES))

In [ ]:
# # print(y_pred_labels[:100])
# # print(y_true[:100])
# # print confusion matrix using sklearn
# cm = confusion_matrix(y_true, y_pred_labels, labels=CLASS_NAMES)

# from sklearn.metrics import ConfusionMatrixDisplay
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)
# disp.plot()